<a href="https://colab.research.google.com/github/SebastianWikt/DS4002-P1/blob/main/Project_1_text_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importance of Word Choice in Education

### Group 5: Neha Channamraju (leader), Sebastian Wiktorowicz, Catherine Young

In [3]:
! git clone https://github.com/SebastianWikt/DS4002-P1.git

Cloning into 'DS4002-P1'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 32.31 KiB | 3.23 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [5]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [6]:
data = pd.read_csv('/content/DS4002-P1/project_1_dataset.csv')

In [7]:
data.head()

,Global Business School Ranking 2020,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Core Values,Unnamed: 61,Web-Link of the Organization's Information obtained,Unnamed: 63
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Values,No. of words,NaN,NaN
1,﻿Rank in 2020,Rank in 2019,Rank in 2018,3 year average rank,School name,Country,Audit year,Salary today (US$),Weighted salary (US$),Salary percentage increase,...,Less than 250 words in length,Inspiring,Socially responsible,Environmentally responsible,Reconciliatory,Enduring,NaN,NaN,NaN,NaN
2,1,2,5,3,Harvard Business School,US,2018,"208,023","210,110",110,...,Y,N,N,N,N,N,"Respect, Honesty, Accountability",3,https://www.hbs.edu/about/Pages/mission.aspx,NaN
3,2,4,3,3,University of Pennsylvania: Wharton,US,2018,"217,141","211,543",107,...,Y,Y,Y,N,Y,Y,"Tradition, teamwork and collaboration, prof...",9,https://executiveeducation.wharton.upenn.edu/a...,NaN
4,3,1,1,2,Stanford Graduate School of Business,US,2015,"225,589","222,625",117,...,Y,Y,Y,N,Y,Y,"integrity, diversity, respect, freedom of i...",11,https://www.gsb.stanford.edu/experience/about,NaN
